In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import datetime as dt
import datetime

In [2]:
# load the cleaned superstore dataset
superstore = pd.read_excel('dataset/superstore_cleaned.xlsx')

# Market Basket Analysis

Market Basket Analysis is a powerful tool for translating vast amounts of customer transaction and viewing data into simple rules for product promotion and recommendation. These rules are called association and dissociation rules. Association rules contain `"antecedent"` and `"consequent"` e.g. *if {friend chicken} then {ketchup}*, here "friend chicken" is the `"antecedent"` and ketchup is `"consequent"`. From these rules we can identify products frequently purchased together or not. In sum, Association rules tell us that two or more items are related or not.
- {antecedent} &rarr; {consequent} is translated as if {antecedent} then {consequent}
- {X} &rarr; {Y} is translated as if {X} then {Y} 

The main problem in Market Basket Analysis is taking an enormous set of potential association rules and selecting only those which are fruitful for a specific business application (The amount of rules increases exponentially with the number of items). But how do we know which association rules is fruitful? We can use some metrics to assess and quantify the score of association rules. Here are some of commonly used metrics:
- `Support`: an indication of how frequently the item X or (X & Y) appears in the data set.
$$Support(X) = \frac{Frequency(X)}{N}, range: [0,1]$$
$$Support(X \rightarrow Y) = \frac{Frequency(X \& Y)}{N}, range: [0,1]$$
- `Confidence`: tells us the probability that we'll purchase Y, given that we have purchased X.
$$Confidence(X \rightarrow Y) = \frac{Support(X \rightarrow Y)}{Support(X)}, range: [0,1]$$
- `Lift`: tells us that two items (X & Y) occur in transactions together more often than we would expect based on their individual support values. (This means that the relationship is unlikely to be explained by random chance)
    - If `Lift` > 1 then the association rules did not arise by random chance.
$$Lift(X \rightarrow Y) = \frac{Support(X \rightarrow Y)}{Support(X)Support(Y)}, range: [0,\infty]$$
- `Leverage`: Similar to `Lift` but easier to interpret.
    - If `Leverage` > 0 then the association rules did not arise by random chance.
$$Leverage(X \rightarrow Y) = Support(X \& Y) - Support(X)Support(Y), range: [-1,1]$$
- `Zhang` Introduced by Zhang (2000): measures both association and dissociation.
    - Value of +1 indicates perfect association.
    - Value of -1 indicates perfect dissociation.
$$Zhang(X \rightarrow Y) = $$
$$\frac{Support(X \& Y) - Support(X)Support(Y)}{Max[Support(X \& Y)(1 - Support(X)), Support(X)(Support(B) - Support(X \& Y))]}, range: [-1,1]$$


One of the most popular algorithm to use in Market Basket Analysis is Apriori Algorithm. This algorithm is fully explained in this article [(source)](https://pub.towardsai.net/association-discovery-the-apriori-algorithm-28c1e71e0f04). In sum, this algorithm could prune not important rules and select the only important rules by filtering the association rules metrics.

## Prepare the transactions data

Prepare the superstore dataset so that form a list of lists of transactions and then use `TransactionEncoder` to encode list of lists into one hot encoded transactions. This data format is needed if we want to apply association rules mining using `mlextend` library.

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

def zhangs_rule(rules):
    PAB = rules['support'].copy()
    PA = rules['antecedent support'].copy()
    PB = rules['consequent support'].copy()
    NUMERATOR = PAB - PA*PB
    DENOMINATOR = np.max((PAB*(1-PA).values,PA*(PB-PAB).values), axis = 0)
    return NUMERATOR / DENOMINATOR 

In [4]:
# create list of lits transcations based on Sub-Category
sub_category = superstore.groupby(['Order Date', 'Order ID']).agg(transactions = ('Sub-Category', 'unique'))['transactions']
sub_category = [list(trans) for trans in sub_category]

encoder = TransactionEncoder()
onehot = encoder.fit(sub_category).transform(sub_category)
df_sub_category = pd.DataFrame(onehot, columns = encoder.columns_)
df_sub_category

,Accessories,Appliances,Art,Binders,Bookcases,Chairs,Copiers,Envelopes,Fasteners,Furnishings,Labels,Machines,Paper,Phones,Storage,Supplies,Tables
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5004,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False
5005,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
5006,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5007,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False


## Analyze the Sub-Category

Let's set the study case first!
- Let's say that we want transactions of sub-category that appears minimal `2` times in a month.
- Hence, we want the transactions to appear `2 * 12 = 24` times in a year.
- We have 4 year of transactions, So the `min_support` we should consider that is `24 * 4 (years) / 5009 (total trx) = 0.019 ~ 0.02`

In [5]:
# set the minimal transactions appeared in a month
transactions_per_month = 2
min_support = transactions_per_month * 12 * 4 / 5009
print('min_support:', min_support)

# prune the items that less than the 'min_support' threshold
frequent_itemsets = apriori(df_sub_category, min_support = min_support, use_colnames = True, max_len = 2)                           

# compute association rules metrics
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 0) # no pruning
rules['zhang'] = zhangs_rule(rules)
rules

min_support: 0.01916550209622679


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
0,(Binders),(Accessories),0.262727,0.143342,0.032142,0.122340,0.853486,-0.005518,0.976071,-0.188863
1,(Accessories),(Binders),0.143342,0.262727,0.032142,0.224234,0.853486,-0.005518,0.950381,-0.166937
2,(Accessories),(Furnishings),0.143342,0.175085,0.022959,0.160167,0.914797,-0.002138,0.982237,-0.098061
3,(Furnishings),(Accessories),0.175085,0.143342,0.022959,0.131129,0.914797,-0.002138,0.985944,-0.101452
4,(Accessories),(Paper),0.143342,0.237772,0.030545,0.213092,0.896203,-0.003538,0.968637,-0.119097
5,(Paper),(Accessories),0.237772,0.143342,0.030545,0.128463,0.896203,-0.003538,0.982928,-0.131905
6,(Phones),(Accessories),0.162507,0.143342,0.023158,0.142506,0.994169,-0.000136,0.999025,-0.006955
7,(Accessories),(Phones),0.143342,0.162507,0.023158,0.161560,0.994169,-0.000136,0.998870,-0.006800
8,(Storage),(Accessories),0.155121,0.143342,0.020563,0.132561,0.924789,-0.001672,0.987572,-0.087807
9,(Accessories),(Storage),0.143342,0.155121,0.020563,0.143454,0.924789,-0.001672,0.986379,-0.086704


### Case 1: Promote high price products with discount together with the most frequent purchased cheap products

We see that the most frequent `Sub-Category` transactions in Superstore are from `Binders` and `Paper`. Since those products are probably considered as cheap products, we can prepare discount schemes on the expensive consequent items on purchasing of antecedent items (`Binders` and `Paper`) in order to increase the sales on consequent items.

In [6]:
# get 'Binders' as antecedents
antecedents_binders = rules[rules['antecedents'] == {'Binders'}]
antecedents_binders['antecedents'] = antecedents_binders['antecedents'].apply(lambda a: ','.join(list(a)))
antecedents_binders['consequents'] = antecedents_binders['consequents'].apply(lambda a: ','.join(list(a)))

# sort by 'lift' and show top 3
cm = sns.light_palette("green", as_cmap = True)
antecedents_binders.sort_values(by = 'lift', ascending = False).head(3).style.background_gradient(cmap = cm)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
10,Binders,Appliances,0.262727,0.090038,0.025953,0.098784,1.097140,0.002298,1.009705,0.120090
33,Binders,Storage,0.262727,0.155121,0.039728,0.151216,0.974826,-0.001026,0.995399,-0.033841
30,Binders,Phones,0.262727,0.162507,0.039728,0.151216,0.930516,-0.002967,0.986697,-0.091968


In [7]:
# get 'Paper' as antecedents 
antecedents_paper = rules[rules['antecedents'] == {'Paper'}]
antecedents_paper['antecedents'] = antecedents_paper['antecedents'].apply(lambda a: ','.join(list(a)))
antecedents_paper['consequents'] = antecedents_paper['consequents'].apply(lambda a: ','.join(list(a)))

# sort by 'lift' and show top 3
cm = sns.light_palette("green", as_cmap = True)
antecedents_paper.sort_values(by = 'lift', ascending = False).head(3).style.background_gradient(cmap = cm)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
12,Paper,Appliances,0.237772,0.090038,0.021761,0.091520,1.016458,0.000352,1.001631,0.021242
37,Paper,Chairs,0.237772,0.114993,0.026552,0.111671,0.971110,-0.000790,0.996260,-0.037564
47,Paper,Storage,0.237772,0.155121,0.035536,0.149454,0.963470,-0.001347,0.993338,-0.047385


Insights:
- We find that {Binders} &rarr; {Appliances} and {Paper} &rarr; {Appliances} are useful assocation rules (*the highest lift, leverage, conviction, and zhang metrics*). If someone purchases `Binders` or `Paper`, they are likely to purchase it together with `Appliances`.
- From this we can use `Binders` or `Paper` as the antecedents to promote `Appliances`. So every time someone purchases `Binders` or `Paper` (which is most frequent to happen), we can promote `Appliances` and give reasonable discount to the product.
- Interestingly, we know that `Binders` and `Paper` are the most frequent transactions that appears in Superstore, but they have dissociation (*negative zhang value*). Hence `Paper` and `Binders` are not good for cross-selling.

### Case 2: Increase the sales on profitable products by putting them together with their complementary products

Suppose we want to increase the sales on `Phones` since it gives relatively high total profit and high amount of transactions (see the EDA section). What are the products that are frequently purchased together with `Phones` so that we can put those products close to each other and increase the odds of being purchased together?

In [8]:
# get 'Phones' as consequents
consequents_phones = rules[rules['consequents'] == {'Phones'}]
consequents_phones['antecedents'] = consequents_phones['antecedents'].apply(lambda a: ','.join(list(a)))
consequents_phones['consequents'] = consequents_phones['consequents'].apply(lambda a: ','.join(list(a)))

# sort by 'lift'
cm = sns.light_palette("green", as_cmap = True)
consequents_phones.sort_values(by = 'lift', ascending = False).style.background_gradient(cmap = cm)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
20,Art,Phones,0.145937,0.162507,0.024755,0.169631,1.043833,0.001040,1.008578,0.049168
41,Furnishings,Phones,0.175085,0.162507,0.029547,0.168757,1.038458,0.001094,1.007518,0.044894
7,Accessories,Phones,0.143342,0.162507,0.023158,0.161560,0.994169,-0.000136,0.998870,-0.006800
30,Binders,Phones,0.262727,0.162507,0.039728,0.151216,0.930516,-0.002967,0.986697,-0.091968
48,Storage,Phones,0.155121,0.162507,0.023358,0.150579,0.926598,-0.001850,0.985957,-0.085723
45,Paper,Phones,0.237772,0.162507,0.034937,0.146935,0.904176,-0.003703,0.981746,-0.122067


Insights:
- We find that `Phones` are frequently purchased together with `Art` or `Furnishings` as the antecedents. This is not due to random chance (*lift > 1, also zhang's and leverage's values are positive*). This may indicate that `Phones` is the complementary product for `Art` and `Furnishings`.
- We could increase the sales on `Phones` by putting `Phones` products at a place that closer to `Art` or `Furnishings` products. This will increase the probability of purchasing `Phones` when the customers purchase `Art` or `Furnishings`.

### Case 3: Bundle most frequent purchased products

Let's change the minimal transcations of `Sub-Category` that appears in a month, that is only 1 transactions in a month. So the `min_support = 1 * 12 * 4/ 5009 = 0.00958`. We also only want the purchased products are 3 `Sub-Category` in total, then set `max_len = 3` and filter it.

In [9]:
# set the minimal transactions appeared in a month
transactions_per_month = 1
min_support = transactions_per_month * 12 * 4 / 5009
print('min_support:', min_support)

# prune the items that less than the 'min_support' threshold
frequent_itemsets = apriori(df_sub_category, min_support = min_support, use_colnames = True, max_len = 3)
                            
# compute association rules metrics
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 0) # no pruning
rules['zhang'] = zhangs_rule(rules)
rules['length'] = rules.apply(lambda x: len(x['antecedents']) + len(x['consequents']), axis = 1)
rules['antecedents'] = rules['antecedents'].apply(lambda a: ','.join(list(a)))
rules['consequents'] = rules['consequents'].apply(lambda a: ','.join(list(a)))

# filter the association rules so that contains only 3 purchased products
rules = rules[rules['length'] > 2]

# sort by 'lift'
cm = sns.light_palette("green", as_cmap = True)
rules.sort_values('lift', ascending = False).style.background_gradient(cmap = cm)

min_support: 0.009582751048113396


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang,length
108,"Binders,Furnishings",Storage,0.039529,0.155121,0.009583,0.242424,1.562810,0.003451,1.115241,0.374948,3
109,Storage,"Binders,Furnishings",0.155121,0.039529,0.009583,0.061776,1.562810,0.003451,1.023712,0.426246,3
111,Furnishings,"Storage,Binders",0.175085,0.039728,0.009583,0.054732,1.377652,0.002627,1.015872,0.332310,3
106,"Storage,Binders",Furnishings,0.039728,0.175085,0.009583,0.241206,1.377652,0.002627,1.087140,0.285469,3
107,"Storage,Furnishings",Binders,0.027550,0.262727,0.009583,0.347826,1.323906,0.002345,1.130485,0.251591,3
110,Binders,"Storage,Furnishings",0.262727,0.027550,0.009583,0.036474,1.323906,0.002345,1.009262,0.331844,3
121,Storage,"Binders,Paper",0.155121,0.054901,0.010581,0.068211,1.242434,0.002065,1.014284,0.230954,3
120,"Binders,Paper",Storage,0.054901,0.155121,0.010581,0.192727,1.242434,0.002065,1.046585,0.206463,3
113,"Binders,Paper",Phones,0.054901,0.162507,0.010781,0.196364,1.208336,0.001859,1.042129,0.182431,3
116,Phones,"Binders,Paper",0.162507,0.054901,0.010781,0.066339,1.208336,0.001859,1.012251,0.205871,3


Insights:
- We have some useful association rules of customers purchasing patterns with 3 products.
- From the strongest association rules, the highest lift, a customer who purchases `Binders` and `Furnishings` together is likely to purchase `Storage` with 24.2% probability. This probability is relatively high.
- We could put `Binders`, `Furnishings`, and `Storage` as a single bundle of product with a lower price compare to each price combined to increase the sales.

# Conclusions

Here are some conclusion to sum up what we have been through so far.
1. The main driver of loss in Superstore is due to high discount given. We clearly see that the more discounts have been offered, the lesser profits have achieved. Products with no discounts show high range of profits but as the discount range increases, we only see more and more loss with hardly any profit. In fact, when the discount given is exceeding 20%, there are no more profits made by Superstore. The most profitable region so far is the Western of United States. 
2. We have used three machine learning algorithms to perform customer segmentation using LRFM model metrics. The algorithm we will implement is K-Means clustering since it gives us the highest evaluation metric scores (DB index and Silhouette score) compared to the other two algorithms. From K-Means clustering we get 4 segments (793 total customers), i.e.
    - `"Lost"` customer segment, 13.11% from total customers. They are relatively old customers (L = 2) but their last purchases are very long ago (R = 1),  purchased very few (F = 1), and spent the lowest (M = 1) compare to other segments. (`LRFM = 2111`)
    - `"Important"` customer segment, 51.32% from total customers. They are the oldest customers (L = 4), purchased most recently (R = 4) but not the most frequently (F = 3) and spent quite a lot (M = 3). (`LRFM = 4433`)
    - `"Champion"` customer segment, 19.55% from total customers. They are commonly old customers (L = 3) who have purchased quite recently (R = 3), purchased most frequently (F = 4) and spent the most (M = 4). (`LRFM = 3344`)
    - `"Potential"` customer segment, 16.0% from total customers. They have good amount of spending (M = 2) and purchases (F = 2) although they are fresh customers (L = 1) but their last purchase are quite a long ago (R = 2). (`LRFM = 1222`)
3. We have used Apriori algorithm to get some insights about purchasing patterns of customers. Those purchasing patterns are in the form of strong association rules i.e. 
    - {Binders} &rarr; {Appliances} and {Paper} &rarr; {Appliances}.
    - {Art} &rarr; {Phones} and {Furnishings} &rarr; {Phones}.
    - {Binders, Furnishings} &rarr; {Storage}.
    
   The results of this market basket analysis can be then used for a data-driven marketing strategy and decision making for product recommendation.

# Recommendations

1. Superstore has to review and evaluate the marketing strategy of offering high discount to a product since that will lead to a loss.
2. "Consumer" customer is the most profitable then followed by "Corporate" customer. The marketing strategy should always focus on retaining this two customers. For "Home Office" customer, these customers might be busy with their work and less likely to spend their time selecting individual products. Hence create a special "Home Office package" with products used for offices for them.
3. Top 5 most profitable sub-category are "Copiers", "Phones", "Accessories", "Paper", and "Binders". The marketing strategy has to focus on marketing these products, especially for high demand products, "Paper" and "Binders". 
   -  "Paper" and "Binders" are good for cross-selling strategy or bundling products to increase the profit, sales, and demand on low performance sub-category. In fact, we discover from market basket analysis that "Binders" or "Paper" are most likely purchased together with "Appliances". Hence, every time someone purchases "Binders" or "Paper" (which is most frequent to happen), we can offer "Appliances" and give reasonable discount to attract more sales.
   - "Phones" is a sub-category with the highest sales. We also see that "Art" or "Furnishings" are frequently purchased together with "Phones". Therefore, we could increase the sales on "Phones" by putting "Phones" products at a place that closer to "Art" or "Furnishings" products. This will increase the probability of purchasing "Phones" when the customers purchase "Art" or "Furnishings".
   - Create a single bundle of product with a lower price compare to each price combined for sub categories of "Binders", "Furnishings", and "Storage". This way will attract more sales and generate more income since customers who purchase "Binders" and "Furnishings" together are more likely to purchase it together with "Storage".
4. Sub category such as "Tables", "Bookcases", and "Machines" are the loss-making sub-category. Those sub-categories relatively have high average discount and low total quantity (in term of transaction too). This might indicate that the marketing strategy of giving high average discount to a low demand product to increase the sales and transactions didn't help that much. Superstore should consider another marketing strategy such as, 
    - Bundle or package them together with high selling products to create "Office Furniture Package" ("Chairs", "Tables", and "Bookcases") to offset the losses,
    - Change suppliers or bargain for cheaper price, or
    - Remove these sub-categories since they might not fit for the customer (also for "Fasteners" and "Supplies").
5. Superstore has to ensure their stock for "Office Supplies" category is always well-stocked and available over time since it is the category with the highest total transactions and quantities, especially for "Binders" and "Papers" sub-category (those are likely to be the most purchased categories every month). There is also a seasonality pattern at the end of the month where the highest total transactions and quantities has been occurring there for 4 years, they need to increase their stock for the most purchased products, so there would not be understocked product at the end of the month since over the year the trend of sales is increasing.
6. Some states have fewer in term of sales and transactions, lack of awareness can be the reason for this, hence advertising in those states might help in more sales.
7. Recommendation for segmented customers:
    - `"Champion"` customer segment: This is the most valuable customer. Focus to improve their frequency and retention such as loyalty programs, give rewards to make them feel respected, market our most expensive products on these customers, offer new products, and cross-selling/up-selling strategy.
    - `"Important"` customer segment: This is loyal customer. Focus to maintain their loyalty and improve their value such as offer a recommendation of some new products and try to give a price incentives on expensive products or more competitive price.
    - `"Potential"` customer segment: This has a potential to be a valuable customer. Focus to improve their value, retention, and frequency such as cross-selling/up-selling strategy, give price incentives and new products recomendation.
    - `"Lost"` customer segment: This customer have already churned. Focus to reactivate the customer by forming a reactivation strategy such as send them reactivation emails, make limited time offers, and ask them for feedback.

# References 

1. [A Case Study of Applying Customer Segmentation in A Medical Equipment Industry](https://iptek.its.ac.id/index.php/jps/article/view/11139/6228)
2. [Customer Segmentation to Identify Key Customers Based on RFM Model by Using Data Mining Techniques](http://www.riejournal.com/article_138379.html)
3. [How RFM Analysis Boosts Sales](https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales)
4. [Build Better and Accurate Clusters with Gaussian Mixture Models](https://www.analyticsvidhya.com/blog/2019/10/gaussian-mixture-models-clustering/#h2_1)
5. [Customer Segmentation: Unsupervised Machine Learning Algorithms In Python](https://towardsdatascience.com/customer-segmentation-unsupervised-machine-learning-algorithms-in-python-3ae4d6cfd41d)
6. [Market Basket Analysis](https://medium.com/nerd-for-tech/market-basket-analysis-1c38613fdd6b)
7. [How To Perform Market Basket Analysis in Python](https://medium.com/@jihargifari/how-to-perform-market-basket-analysis-in-python-bd00b745b106)